In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls drive/MyDrive/Classes/ece542/competition-project/

 2_conv-plus-normalization.ipynb
 attempt-1.ipynb
 Best
 best-conv-plus-normalization.ipynb
 Best_Models
'Copy of attempt-1.ipynb'
'Copy of conv-plus-normalization.ipynb'
'Copy of current-conv-plus-normalization.ipynb'
'Copy of Hyoer attempt-1.ipynb'
 current-conv-plus-normalization.ipynb
 data.zip
 experiments
'Hyoer attempt-1.ipynb'
 lstm
 ml_utils
 Prediction
 test_data.zip
 window3


In [ ]:
!cp drive/MyDrive/Classes/ece542/competition-project/window3/window3_data.zip .

In [ ]:
!unzip window3_data.zip

Archive:  window3_data.zip
   creating: window3_splits/test/
  inflating: window3_splits/test/subject_005_session_03__x.csv  
  inflating: window3_splits/test/subject_005_session_03__y.csv  
  inflating: window3_splits/test/subject_006_session_01__x.csv  
  inflating: window3_splits/test/subject_006_session_01__y.csv  
  inflating: window3_splits/test/subject_006_session_02__x.csv  
  inflating: window3_splits/test/subject_006_session_02__y.csv  
  inflating: window3_splits/test/subject_006_session_03__x.csv  
  inflating: window3_splits/test/subject_006_session_03__y.csv  
  inflating: window3_splits/test/subject_007_session_01__x.csv  
  inflating: window3_splits/test/subject_007_session_01__y.csv  
  inflating: window3_splits/test/subject_007_session_02__x.csv  
  inflating: window3_splits/test/subject_007_session_02__y.csv  
  inflating: window3_splits/test/subject_007_session_03__x.csv  
  inflating: window3_splits/test/subject_007_session_03__y.csv  
  inflating: window3_splits/t

In [ ]:
!rm window3_data.zip

In [ ]:
# Access token redacted. Replace <ACCESS_TOKEN> with your access token.
!git clone https://savadikarc:<ACCESS_TOKEN>@github.com/savadikarc/ece542-competition-project.git

Cloning into 'ece542-competition-project'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (238/238), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 238 (delta 128), reused 158 (delta 60), pack-reused 0
Receiving objects: 100% (238/238), 275.09 KiB | 7.24 MiB/s, done.
Resolving deltas: 100% (128/128), done.


In [ ]:
!cd ece542-competition-project && git checkout phase-1 && git pull origin phase-1

Already on 'phase-1'
Your branch is up to date with 'origin/phase-1'.
From https://github.com/savadikarc/ece542-competition-project
 * branch            phase-1    -> FETCH_HEAD
Already up to date.


In [ ]:
base_path = "drive/MyDrive/Classes/ece542/competition-project"

In [ ]:
import os
import sys
sys.path.append("ece542-competition-project")

In [ ]:
import json
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from torch.utils.data.sampler import WeightedRandomSampler
import numpy as np
from ml_utils.dataset import SubjectDataset

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cpu


In [ ]:
base_data_path = os.path.join("window3_splits")
train_data_path = os.path.join(base_data_path, "train")
val_data_path = os.path.join(base_data_path, "val")

splits_file = os.path.join(base_path, "experiments", "metadata", "split_ids.json")
with open(splits_file, "r") as f:
    split_ids = json.load(f)

In [ ]:
stats_path = os.path.join(base_path, "experiments", "metadata", "statistics.json")
with open(stats_path, "r") as f:
    stats = json.load(f)

In [ ]:
min = np.array([v["min"] for k, v in stats.items()])
max = np.array([v["max"] for k, v in stats.items()])

min = torch.from_numpy(min).float()
min = torch.unsqueeze((torch.unsqueeze(min, 0)), -1)
min = min.to(device)
max = torch.from_numpy(max).float().to(device)
max = torch.unsqueeze((torch.unsqueeze(max, 0)), -1)
max = max.to(device)

In [ ]:
min.shape

torch.Size([1, 6, 1])

In [ ]:
# Training hyperparameters
batch_size = 128
num_epochs = 30

In [ ]:
train_dataset = SubjectDataset(
    train_data_path, 
    split_ids["train"]
)
ys = train_dataset.y.tolist()
counts = Counter(ys)
weights = np.array([1./counts[_y] for _y in ys])
sample_weights = torch.from_numpy(weights).float()
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),
    replacement=True)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, sampler=sampler)
train_iterations = (len(train_dataset) // batch_size) + ((len(train_dataset) % batch_size) != 0)

val_dataset = SubjectDataset(
    val_data_path, 
    split_ids["val"]
)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)
val_iterations = (len(val_dataset) // batch_size) + ((len(val_dataset) % batch_size) != 0)

Converting uid 005_02
Converting uid 001_06
Converting uid 003_02
Converting uid 001_05
Converting uid 002_02
Converting uid 003_01
Converting uid 003_03
Converting uid 005_01
Converting uid 001_07
Converting uid 002_05
Converting uid 004_02
Converting uid 002_03
Converting uid 001_02
Converting uid 002_04
Converting uid 001_03
Converting uid 004_01
Converting uid 001_08
Converting uid 002_01
Converting uid 001_01
Converting uid 001_04


In [ ]:
print(train_iterations)
print(val_iterations)

1373
422


In [ ]:
class OneDConvNet(nn.Module):
  def __init__(self, n_features, n_classes, base_filters=32):
    super(OneDConvNet, self).__init__()

    self.conv1 = nn.Conv1d(in_channels=n_features, out_channels=base_filters, kernel_size=3, stride=1, padding=1)
    self.norm1 = nn.LayerNorm(base_filters*120)
    self.pool1 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv2 = nn.Conv1d(in_channels=base_filters, out_channels=base_filters*2, kernel_size=3, stride=1, padding=1)
    self.norm2 = nn.LayerNorm(base_filters*2*60)
    self.pool2 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv3 = nn.Conv1d(in_channels=base_filters*2, out_channels=base_filters*4, kernel_size=3, stride=1, padding=1)
    self.norm3 = nn.LayerNorm(base_filters*4*30)
    self.pool3 = nn.AvgPool1d(kernel_size=2, stride=2)
    self.conv4 = nn.Conv1d(in_channels=base_filters*4, out_channels=base_filters*8, kernel_size=3, stride=1, padding=1)
    self.norm4 = nn.LayerNorm(base_filters*8*15)
    self.fc1 = nn.Linear(base_filters*8*15, base_filters*16)
    self.dropout5 = nn.Dropout(0.5)
    self.fc2 = nn.Linear(base_filters*16, n_classes)
  
  def forward(self, x):
    x = self.conv1(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm1(x)
    x = x.view(N, C, T)
    x = self.pool1(F.relu(x))

    x = self.conv2(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm2(x)
    x = x.view(N, C, T)
    x = self.pool2(F.relu(x))

    x = self.conv3(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm3(x)
    x = x.view(N, C, T)
    x = self.pool3(F.relu(x))

    x = self.conv4(x)
    N, C, T = x.shape
    x = x.view(N, C*T)
    x = self.norm4(x)
    x = x.view(N, C, T)
    x = F.relu(x)

    N, C, T = x.size()
    x = x.view(-1, C*T) # Flatten
    x = self.dropout5(F.relu(self.fc1(x)))
    x = self.fc2(x)
    return x

In [ ]:
base_filters = 32
interval = 3
checkpoint_save_path = os.path.join(base_path, "experiments", "models", "window3")
if not os.path.exists(checkpoint_save_path):
    os.makedirs(checkpoint_save_path)
filename = f"cnn_{base_filters}_interval{interval}.pth"
filename_long = f"cnn_{base_filters}_interval{interval}_long.pth"
model = OneDConvNet(6, 4, base_filters).to(device)

In [ ]:
print(model)

OneDConvNet(
  (conv1): Conv1d(6, 32, kernel_size=(3,), stride=(1,), padding=(1,))
  (norm1): LayerNorm((3840,), eps=1e-05, elementwise_affine=True)
  (pool1): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (conv2): Conv1d(32, 64, kernel_size=(3,), stride=(1,), padding=(1,))
  (norm2): LayerNorm((3840,), eps=1e-05, elementwise_affine=True)
  (pool2): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (conv3): Conv1d(64, 128, kernel_size=(3,), stride=(1,), padding=(1,))
  (norm3): LayerNorm((3840,), eps=1e-05, elementwise_affine=True)
  (pool3): AvgPool1d(kernel_size=(2,), stride=(2,), padding=(0,))
  (conv4): Conv1d(128, 256, kernel_size=(3,), stride=(1,), padding=(1,))
  (norm4): LayerNorm((3840,), eps=1e-05, elementwise_affine=True)
  (fc1): Linear(in_features=3840, out_features=512, bias=True)
  (dropout5): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=512, out_features=4, bias=True)
)


In [ ]:
import numpy as np
X = np.random.randn(1, 6, 120)
T = torch.from_numpy(X).float().to(device)
y = model(T)

In [ ]:
# y.shape

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)
# optimizer = optim.Adam(model.parameters(), lr=1e-4)
# optimizer = optim.RMSprop(model.parameters(), lr=1e-4)
scheduler = ReduceLROnPlateau(optimizer, factor=0.5, patience=1, verbose=True)
criterion = nn.CrossEntropyLoss()

In [ ]:
def train_step(X, y, model, optimizer, criterion):

    y_pred = model(X)
    predicted_classes = torch.argmax(y_pred.detach(), dim=1)

    loss = criterion(y_pred, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    corrects = torch.sum(y.data == predicted_classes)

    return loss.item(), corrects

def val_step(X, y, model, criterion):

    with torch.no_grad():
        
        y_pred = model(X)
        predicted_classes = torch.argmax(y_pred.detach(), dim=1)
        loss = criterion(y_pred, y)
        corrects = torch.sum(y.data == predicted_classes)

    return loss.item(), corrects, predicted_classes.detach().cpu().numpy()

In [ ]:
best_val_loss = float("inf")

for epoch in range(num_epochs):
    # Train for "n" number of iterations
    running_loss = 0.
    running_acc = 0.
    for iteration, (X, y) in enumerate(train_dataloader):

        X = X.float().to(device)
        # Normalize
        X = (X - min) / (max - min)

        y = y.view(X.size(0)).to(device)

        loss, corrects = train_step(X, y, model, optimizer, criterion)

        # Running metrics
        running_loss = running_loss + loss * X.size(0)
        running_acc = running_acc + corrects

        if iteration % 100 == 0:
            print(f"Iteration: {iteration}/{train_iterations} | train_loss: {loss} | train_acc: {corrects/X.size(0)}")

    train_loss = running_loss / len(train_dataset)
    train_acc = running_acc / len(train_dataset)

    # Validate
    running_val_loss = 0.
    running_val_acc = 0.
    for step, (X, y) in enumerate(val_dataloader):

        X = X.float().to(device)
        X = (X - min) / (max - min)

        y = y.view(X.size(0)).to(device)

        loss, corrects, predicted_classes = val_step(X, y, model, criterion)
        # Running metrics
        running_val_loss = running_val_loss + loss * X.size(0)
        running_val_acc = running_val_acc + corrects

    val_loss = running_val_loss / len(val_dataset)
    val_acc = running_val_acc / len(val_dataset)

    scheduler.step(val_loss)

    if val_loss < best_val_loss:
        # Checkpoint model
        path = os.path.join(checkpoint_save_path, filename)
        print(f"Saving model to {path}")
        torch.save(model.state_dict(), path)
        best_val_loss = val_loss

    print(f"Epoch: {epoch} | train_loss {train_loss} | train_acc: {train_acc} | val_loss: {val_loss} | val_acc: {val_acc}")

Iteration: 0/1373 | train_loss: 1.4241429567337036 | train_acc: 0.234375
Iteration: 100/1373 | train_loss: 1.3967080116271973 | train_acc: 0.2421875
Iteration: 200/1373 | train_loss: 1.3771915435791016 | train_acc: 0.2734375
Iteration: 300/1373 | train_loss: 1.3846782445907593 | train_acc: 0.2421875
Iteration: 400/1373 | train_loss: 1.3864760398864746 | train_acc: 0.2890625
Iteration: 500/1373 | train_loss: 1.3799875974655151 | train_acc: 0.25
Iteration: 600/1373 | train_loss: 1.361541509628296 | train_acc: 0.3125
Iteration: 700/1373 | train_loss: 1.3636505603790283 | train_acc: 0.34375
Iteration: 800/1373 | train_loss: 1.3502804040908813 | train_acc: 0.359375
Iteration: 900/1373 | train_loss: 1.3550267219543457 | train_acc: 0.375
Iteration: 1000/1373 | train_loss: 1.3322718143463135 | train_acc: 0.46875
Iteration: 1100/1373 | train_loss: 1.299526572227478 | train_acc: 0.5390625
Iteration: 1200/1373 | train_loss: 1.249451994895935 | train_acc: 0.578125
Iteration: 1300/1373 | train_loss

In [ ]:
path = os.path.join(os.path.join(checkpoint_save_path, filename_long))
print(f"Saving model to {path}")
torch.save(model.state_dict(), path)

Saving model to drive/MyDrive/Classes/ece542/competition-project/experiments/models/window3/cnn_32_interval3_long.pth


In [ ]:
best_model = OneDConvNet(6, 4, base_filters)
best_model.load_state_dict(torch.load(os.path.join(checkpoint_save_path, filename)))
best_model = best_model.to(device)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

class Evaluation:

  def __init__(self):
        pass
  
  def aggregate_metrics(self, y, y_hat):
    
    """ Metrics for entire set 
        
        Arguments passed are: 
                      y     : ground truth  
                      y_hat : predicted class
        Calculations:
                  accuracy  : (tp + tn) / (total samples =(p + n))
                  precision : tp / (tp + fp)
                  recall    : tp / (tp + fn)
                  f1        : 2 tp / (2 tp + fp + fn)
    """
    accuracy  = accuracy_score(y, y_hat)
    precision = precision_score(y, y_hat, average="macro")
    recall    = recall_score(y, y_hat, average="macro")
    f1        = f1_score(y, y_hat, average="macro")
    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1": f1
        }

  def classwise_metrics(self, y, y_hat):

    """ Metrics for each class. 
        Average is set to macro for calculating the score of each label, and find their unweighted mean. 
        This does not take label imbalance into account.
        
        Arguments passed are: 
                      y     : ground truth  
                      y_hat : predicted class
        Calculations:
                  accuracy  : (tp + tn) / (total samples =(p + n))
                  precision : tp / (tp + fp)
                  recall    : tp / (tp + fn)
                  f1        : 2 tp / (2 tp + fp + fn)
    """
    
    class_precision = precision_score(y, y_hat, average=None)
    class_recall    = recall_score(y, y_hat, average=None)
    class_f1        = f1_score(y, y_hat, average=None)
    
    return {
        "Precision": class_precision,
        "Recall": class_recall,
        "F1": class_f1
    }

In [ ]:
evaluator = Evaluation()

In [ ]:
val_dataset = SubjectDataset(
    val_data_path, 
    split_ids["val"]
)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
val_iterations = (len(val_dataset) // batch_size) + ((len(val_dataset) % batch_size) != 0)

Converting uid 001_08
Converting uid 002_01
Converting uid 001_01
Converting uid 001_04


In [ ]:
output = []
labels = []
for X, y in val_dataloader:
    X = X.float().to(device)
    X = (X - min) / (max - min)
    
    y = y.view(X.size(0)).to(device)

    y_pred = best_model(X)
    predicted_classes = torch.argmax(y_pred, dim=1).detach().cpu().numpy()
    y_true = y.cpu().numpy()

    output.append(predicted_classes)
    labels.append(y_true)

In [ ]:
_output = np.concatenate(output, axis=0)
_labels = np.concatenate(labels, axis=0)

In [ ]:
agg_metrics = evaluator.aggregate_metrics(_labels, _output)
print("Agg metrics")
print(agg_metrics)
classwise_metrics = evaluator.classwise_metrics(_labels, _output)
print("Classwise Metrics")
print(classwise_metrics)

Agg metrics
{'Accuracy': 0.8392111756302989, 'Precision': 0.7669642439415235, 'Recall': 0.843616337791005, 'F1': 0.7913423483641792}
Classwise Metrics
{'Precision': array([0.87514906, 0.73543092, 0.59547901, 0.86179799]), 'Recall': array([0.88902975, 0.91149542, 0.91196835, 0.66197183]), 'F1': array([0.8820348 , 0.81405209, 0.72050013, 0.74878238])}


In [ ]:
test_data_path = os.path.join(base_data_path, "test")
test_dataset = SubjectDataset(
    test_data_path, 
    split_ids["test"]
)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
test_iterations = (len(test_dataset) // batch_size) + ((len(test_dataset) % batch_size) != 0)

Converting uid 005_03
Converting uid 006_01
Converting uid 006_02
Converting uid 006_03
Converting uid 007_01
Converting uid 007_02
Converting uid 007_03
Converting uid 007_04
Converting uid 008_01


In [ ]:
output = []
labels = []
for X, y in test_dataloader:
    X = X.float().to(device)
    X = (X - min) / (max - min)
    
    y = y.view(X.size(0)).to(device)

    y_pred = best_model(X)
    predicted_classes = torch.argmax(y_pred, dim=1).detach().cpu().numpy()
    y_true = y.cpu().numpy()

    output.append(predicted_classes)
    labels.append(y_true)

In [ ]:
_output = np.concatenate(output, axis=0)
_labels = np.concatenate(labels, axis=0)

In [ ]:
agg_metrics = evaluator.aggregate_metrics(_labels, _output)
print("Agg metrics")
print(agg_metrics)
classwise_metrics = evaluator.classwise_metrics(_labels, _output)
print("Classwise Metrics")
print(classwise_metrics)

Agg metrics
{'Accuracy': 0.8832410050859314, 'Precision': 0.7872710426580869, 'Recall': 0.9150362826641119, 'F1': 0.8411878791012724}
Classwise Metrics
{'Precision': array([0.97286864, 0.74481013, 0.7808265 , 0.6505789 ]), 'Recall': array([0.87901844, 0.94689411, 0.96585265, 0.86837993]), 'F1': array([0.92356547, 0.83378206, 0.86353964, 0.74386435])}
